In [1]:
!pip install --upgrade transformers
!pip install tokenizer
!pip install hugchat

In [2]:
!pip install beautifulsoup4

In [1]:
import numpy as np
import pandas as pd

import getpass
from hugchat import hugchat
from hugchat.login import Login

from bs4 import BeautifulSoup as bs
import requests

## What is Web scraping.?¶
Web scraping in Python involves extracting data from websites, and there are several methods and techniques you can use to achieve this goal. Here are some of the different methods of web scraping in Python:

Using BeautifulSoup and Requests: This is one of the most common methods for web scraping in Python. The requests library is used to fetch the HTML content of web pages, and the BeautifulSoup library is used to parse and extract data from the HTML. This method is suitable for static web pages with well-structured HTML. Using Selenium:

Selenium: It is a web automation tool that allows you to control a web browser programmatically. It can be used for web scraping dynamic websites that rely heavily on JavaScript, as it can interact with the page as a human user would. Selenium is often used when other methods fail to retrieve the data you need. Scrapy Framework:

Scrapy: It is a powerful and flexible web scraping framework for Python. It provides a high-level framework for defining how to extract data from websites, follow links, and store the data. Scrapy is well-suited for more complex web scraping projects and can handle crawling multiple pages and websites.

APIs: Many websites offer APIs (Application Programming Interfaces) that allow you to access and retrieve data in a structured format, such as JSON or XML. Instead of scraping HTML, you can make API requests to fetch the data directly. Using APIs is often more efficient and reliable than web scraping HTML.

## Pre-Processing COVID-19 Data
In this section, we will process a covid-19 data from worldometers.info, extracting all the coutry's details like critical cases, recovered cases. and also do preprocessing for each columns as needed.

## Loading the HTML Data of COVID-19 from Worldometers

In [5]:
from bs4 import BeautifulSoup as bs
def scrape_covid_data():
    url = 'https://www.worldometers.info/coronavirus/'
    response = requests.get(url)
    
    if response.status_code == 200:
        global zzz
        zzz = response.content
        soup = bs(response.content, 'html.parser')
        table = soup.find('table', id = 'main_table_countries_today')
        rows = table.find_all('tr')
        return rows

    else:
        print('Failed to fetch data. Status code:', response.status_code)
        return None


if __name__ == '__main__':
    covid_data = scrape_covid_data()
    if covid_data is not None:
        print(covid_data)

# For Writing text file
file1 = open("web_scrape_covid_text.txt","w")
file1.write(str(covid_data))
file1.close()


[<tr>
<th width="1%">#</th>
<th width="100">Country,<br>Other</br></th>
<th width="20">Total<br>Cases</br></th>
<th width="30">New<br>Cases</br></th>
<th width="30">Total<br>Deaths</br></th>
<th width="30">New<br>Deaths</br></th>
<th width="30">Total<br>Recovered</br></th>
<th width="30">New<br>Recovered</br></th>
<th width="30">Active<br/>Cases</th>
<th width="30">Serious,<br/>Critical</th>
<th width="30">Tot Cases/<br/>1M pop</th>
<th width="30">Deaths/<br/>1M pop</th>
<th width="30">Total<br/>Tests</th>
<th width="30">Tests/<br/>
<nobr>1M pop</nobr>
</th>
<th width="30">Population</th>
<th style="display:none" width="30">Continent</th>
<th width="30">1 Case<br/>every X ppl</th><th width="30">1 Death<br/>every X ppl</th><th width="30">1 Test<br/>every X ppl</th>
<th width="30">New Cases/1M pop</th>
<th width="30">New Deaths/1M pop</th>
<th width="30">Active Cases/1M pop</th>
</tr>, <tr class="total_row_world row_continent" data-continent="North America" style="display: none">
<td></t

### Read file using Open function. Which returns String of HTML file

In [8]:
with open('web_scrape_covid_text.txt') as f:
    text = f.read()

In [9]:
print(type(text))

<class 'str'>


#### Here we have created text file from our scrape data so we have to convert it into html parser to access particular rows using tr (table row) tag.

In [14]:
new_soup = bs(text, 'html.parser')
new_rows = new_soup.find_all('tr')

## Preprocessing of Columns

### The country column contains '\n' in it's rows for some values, therefore we are replacing '\n' with a null string "".

In [18]:
rows=new_rows
country_data = []
for row in rows[1:]:
            columns = row.find_all('td')
            country = columns[1].text.replace("\n","")
            total_cases = columns[2].text
            new_cases = columns[3].text.strip()
            total_deaths = columns[4].text
            new_deaths = columns[5].text
            total_recovered = columns[6].text
            active_cases = columns[8].text
            critical_cases = columns[9].text
            total_tests = columns[12].text
            country_data.append({
                'Country': country,
                'Total Cases': total_cases,
                'New Cases': new_cases,
                'Total Deaths': total_deaths,
                'New Deaths': new_deaths,
                'Total Recovered': total_recovered,
                'Active Cases': active_cases,
                'Critical Cases': critical_cases,
                'Total Tests': total_tests
            })

In [20]:
covid_data=pd.DataFrame(country_data)

In [22]:
covid_data

,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,Active Cases,Critical Cases,Total Tests
0,North America,"131,889,132",,"1,695,941",,"127,665,129","2,528,062","6,095",
1,Asia,"221,500,265",,"1,553,662",,"205,673,091","14,273,512","14,733",
2,Europe,"253,406,198",,"2,101,824",,"248,754,104","2,550,270","4,453",
3,South America,"70,200,879",,"1,367,332",,"66,683,585","2,149,962","8,953",
4,Oceania,"14,895,771",,"33,015",,"14,752,388","110,368",31,
...,...,...,...,...,...,...,...,...,...
242,Total:,"70,200,879",,"1,367,332",,"66,683,585","2,149,962","8,953",
243,Total:,"14,895,771",,"33,015",,"14,752,388","110,368",31,
244,Total:,"12,860,924",,"258,892",,"12,090,808","511,224",529,
245,Total:,721,,15,,706,0,0,


#### As we can see New cases and New death columns has null values so, we will remove that two columns.

In [25]:
covid_data.drop(['New Cases','New Deaths','Total Tests'],axis=1,inplace=True)

In [27]:
covid_data.head()

,Country,Total Cases,Total Deaths,Total Recovered,Active Cases,Critical Cases
0,North America,"131,889,132","1,695,941","127,665,129","2,528,062","6,095"
1,Asia,"221,500,265","1,553,662","205,673,091","14,273,512","14,733"
2,Europe,"253,406,198","2,101,824","248,754,104","2,550,270","4,453"
3,South America,"70,200,879","1,367,332","66,683,585","2,149,962","8,953"
4,Oceania,"14,895,771","33,015","14,752,388","110,368",31


#### Countries which names starts with C.

In [30]:
def startswith(column,letter):
    List1=[]
    for i in column:
        if i[:1] == letter:
            List1.append(i)
    return List1

In [32]:
startswith(covid_data['Country'],"I")

['India',
 'Italy',
 'Iran',
 'Indonesia',
 'Israel',
 'Iraq',
 'Ireland',
 'Iceland',
 'Ivory Coast',
 'Isle of Man']

#### Countries which names starts with and ends with A.

In [35]:
import re
def startendwith(column):
    r = re.compile("A|a.*A|a")
    newlist = list(filter(r.match, column)) # Read Note below
    return(newlist)

In [37]:
startendwith(covid_data['Country'])

['Asia',
 'Africa',
 'Australia',
 'Argentina',
 'Austria',
 'Azerbaijan',
 'Armenia',
 'Albania',
 'Algeria',
 'Afghanistan',
 'Angola',
 'Andorra',
 'Aruba',
 'Antigua and Barbuda',
 'Anguilla']

#### Creating a chatbot 

In [42]:
# login to Hugging Face
email = input("Enter your email")
password = getpass.getpass('Enter your password')
sign = Login(email, password)
cookies = sign.login()

# create a chatbot connection using the HuggingChat API
chatbot = hugchat.ChatBot(cookies)

Enter your email shashankbm57@gmail.com
Enter your password ········


#### We will store the chat history in the list chat_history for the messege to the chat bot will be "Can you tell me about this " plus our data

In [43]:
# Initialize an empty list to store chat history
chat_history = []

#  input message containing population_info
message = "Can you tell me about this " +str(covid_data)

# Append the message to the chat history list
chat_history.append(message)

# Join the chat history list into a single string with newline separators
conv = "\n".join(chat_history)

#### Use the chatbot to generate a response based on the conversation history (conv)

In [47]:
ai_response = chatbot.chat(conv)

# Print the AI-generated response
ai_response


#### Append the AI-generated response to the chat history for future reference

In [50]:
chat_history.append(ai_response)

#### The following lines of code will prompt you to ask more questions about your data

In [53]:
# Initialize a flag variable to control the loop
out = False

# Start a loop to continue the conversation
while out == False:  # Using 'while not out' is a cleaner condition
    # Prompt the user to ask more questions or exit
    print("Would you like to know more? Type 'out' to exit")
    print("\n")
    
    # Get user input
    prompt = input()
    
    # Check if the user wants to exit the conversation
    if prompt.lower() == "out":  # Make it case-insensitive
        print("Leaving")
        out = True
        continue  # Skip to the next iteration (exits the loop)
    
    # Join the chat history into a single string
    conv = "\n".join(str(item) for item in chat_history)  # Convert each item to a string
    
    # Generate an AI response based on the conversation history
    ai_response = chatbot.chat(conv)
    
    # Print the AI-generated response
    print(ai_response)
    
    # Append the user's input and AI response to the chat history
    chat_history.append(prompt)
    chat_history.append(ai_response)

    # Check if the user wants to exit the conversation
    if prompt == "out":
        print("Leaving")
        out = True


Would you like to know more? Type 'out' to exit




 out


Leaving
